#  MNIST 

In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/i550012/virtualenv/tensorflow1

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# placeholder
n_pixels = 28*28 # the image has 28*28 pixels   
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, n_pixels], name="x")
    y = tf.placeholder(tf.float32, [None, 10], name="y") # the label is one-hot vector of 10 elements

In [4]:
with tf.name_scope('input_reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image') 


In [5]:
# helper functions

def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape, stddev=0.1, name=name)
    return tf.Variable(initial)

def bias_variable(shape, name=None):
    initial = tf.constant(0.1, shape=shape, name=name)
    return tf.Variable(initial)

def conv(x,W, name=None):   
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME', name=name)

def max_pool(x, name=None):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=name)


In [8]:
# define layers
with tf.name_scope('conv1'):
    with tf.name_scope('weights'):
        W_conv1 = weight_variable([5,5,1,32], name='Weight')
    with tf.name_scope('biases'):
        b_conv1 = bias_variable([32], name='Bias')
    conv1 = conv(x_image, W_conv1, name='Conv1') + b_conv1
    h_conv1 = tf.nn.relu(conv1, name='Relu')
    h_pool1 = max_pool(h_conv1, name='Pool')
    
with tf.name_scope('conv2'):
    with tf.name_scope('weights'):
        W_conv2 = weight_variable([5,5,32,64], name='Weight')
    with tf.name_scope('biases'):
        b_conv2 = bias_variable([64], name='Bias')
    conv2 = conv(h_pool1, W_conv2, name='Conv') + b_conv2
    h_conv2 = tf.nn.relu(conv2, name='Relu')
    h_pool2 = max_pool(h_conv2, name='Pool')


In [9]:
# connect layers
with tf.name_scope('fc'):
    W_c1 = weight_variable([7*7*64, 1024], name='Weight')
    b_c1 = bias_variable([1024], name='Bias')
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_c1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_c1) + b_c1, name='relu')

In [10]:
# dropout
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    h_c1_drop = tf.nn.dropout(h_c1, keep_prob, name='Dropout')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
# readout layer
with tf.name_scope('readout'):
    W_c2 = weight_variable([1024, 10], name='Weight')
    b_c2 = bias_variable([10], name='Bias')

In [12]:
# model
y_conv = tf.matmul(h_c1_drop, W_c2) + b_c2
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1)),tf.float32))

summarize = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.





In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


2024-03-22 09:27:19.302800: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


In [14]:
log_path = 'logs/'
tb = tf.summary.FileWriter(log_path, sess.graph)


In [18]:
# train
for i in range(1500):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0], y:batch[1], keep_prob:0.5})
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
        print("step {}, training accuracy {}".format(i, train_accuracy))

print("accuracy: {0:.3f}%".format(accuracy.eval(feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})))
sess.close()

step 0, training accuracy 0.10000000149011612
step 100, training accuracy 0.8600000143051147
step 200, training accuracy 0.8799999952316284
step 300, training accuracy 0.8399999737739563
step 400, training accuracy 0.8999999761581421
step 500, training accuracy 0.9399999976158142
step 600, training accuracy 0.9399999976158142
step 700, training accuracy 0.9200000166893005
step 800, training accuracy 0.9399999976158142
step 900, training accuracy 0.9800000190734863
step 1000, training accuracy 0.9200000166893005
step 1100, training accuracy 0.9800000190734863
step 1200, training accuracy 0.9599999785423279
step 1300, training accuracy 0.9800000190734863
step 1400, training accuracy 0.9399999976158142
accuracy: 0.970%
